<a href="https://colab.research.google.com/github/shadiakiki1986/ml-competitions/blob/master/replicated_papers/0-WtpRl/WtpDesignerEnv_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install openai gym
!pip install gym | tail

In [2]:
# install rlworkgroup/garage
# Copied from colab/2019-01-21/t3.ipynb
#------------------------------------

# Install dependencies (copied from garage/environment.yml)
!apt-get install libglfw3 libglfw3-dev | tail

# >>>>>>>>   requires restart of runtime in colab.research.google.com due to joblib and rsa <<<<<<
!pip install awscli  boto3  cached_property  cloudpickle  cma==1.1.06 flask  gym  "box2d-py>=2.3.4"  hyperopt  ipdb  ipywidgets  jsonmerge  "joblib<0.13,>=0.12"  jupyter  mako  matplotlib  memory_profiler  pandas  path.py    polling  pre_commit  protobuf  psutil  pygame  pyglet  PyOpenGL  pyprind  python-dateutil  pyzmq  scikit-image  scipy  tensorboard  | tail
#"tensorflow<1.10,>=1.9.0"  Theano==1.0.2    "mujoco-py<1.50.2,>=1.50.1" gym[all]==0.10.8
#!pip install jsonmerge glfw mako pygame
!pip install pyprind cma glfw | tail

# Install garage (continued in next cell)
!git clone https://github.com/rlworkgroup/garage
# !cd garage && pip install -e . # >>>>>>>>   requires restart of runtime in colab.research.google.com due to joblib and rsa <<<<<<
#!pip show rlgarage garage

Preparing to unpack .../5-libxrandr-dev_2%3a1.5.1-1_amd64.deb ...
Unpacking libxrandr-dev:amd64 (2:1.5.1-1) ...
Setting up libvulkan1:amd64 (1.1.82.0-0ubuntu0.18.04.1~gpu1) ...
Setting up libvulkan-dev:amd64 (1.1.82.0-0ubuntu0.18.04.1~gpu1) ...
Setting up libglfw3:amd64 (3.2.1-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up x11proto-randr-dev (2018.4-4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Setting up libglfw3-dev:amd64 (3.2.1-1) ...
Setting up libxrandr-dev:amd64 (2:1.5.1-1) ...
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Found existing installation: botocore 1.12.89
    Uninstalling botocore-1.12.89:
      Successfully uninstalled botocore-1.12.89
  Found existing installation: joblib 0.13.1
    Uninstalling joblib-0.13.1:
      Successfully uninstalled joblib-0.13.1


  Stored in directory: /root/.cache/pip/wheels/c5/53/f9/fd31798dce7e10aa49f8354e4111b9c9cad10c894184658663
Successfully built glfw
Cloning into 'garage'...
remote: Enumerating objects: 347, done.
remote: Counting objects: 100% (347/347), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 10267 (delta 218), reused 218 (delta 153), pack-reused 9920
Receiving objects: 100% (10267/10267), 9.13 MiB | 15.90 MiB/s, done.
Resolving deltas: 100% (7126/7126), done.


In [1]:
# Install garage, commit e7324a68dedd94b4ea15a9c761bab2af032e2480 before the upcoming commits related to bumping gym/dm-control/mujoco versions
!cd garage && git checkout e7324a68dedd94b4ea15a9c761bab2af032e2480
!cd garage && pip install -e . # >>>>>>>>   requires restart of runtime in colab.research.google.com due to joblib and rsa <<<<<<

Note: checking out 'e7324a68dedd94b4ea15a9c761bab2af032e2480'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at e7324a6 Move nb_utils.py to garage.experiment
Obtaining file:///content/garage
  Running setup.py develop for rlgarage




---



---



---



# Start

In [1]:
# Create a gym env that simulates the current water treatment plant
# Based on https://github.com/openai/gym/blob/master/gym/envs/toy_text/nchain.py

import gym
from gym import spaces
#from gym.utils import seeding
import numpy as np
import random



def act_on_water(water_in, water_parameter):
  """
  Parameters
  - water_in: water sample parameters before "filtering"
  - water_parameter: specific parameter that is being filtered
  
  Returns
  - water_out: water sample parameters after the targeted parameter
  - reward:
    - if water quality is improvable and made improvement and improvement is relevant, then +2
    - if water quality is not improvable and did not try to make improvement, then +1
    - if water quality is improvable and made improvement but improvement is not relevant, then 0
    - if water quality is improvable and did not make improvement, then -1
    - if water quality is not improvable and tried to make improvement, then -2
  """
  # debugging
  #print("water_in", water_in, "water_parameter", water_parameter)
  
  # utility variable
  is_improvable = any(water_in[k] for k in water_in)
  
  # if chose pipe (do nothing)
  if water_parameter is None:
    # if any parameter is "high"
    if is_improvable:
      # water was improvable but didn't try
      return water_in, -1
    
    # water was not improvable to begin with
    return water_in, +1

  # sanity check
  if water_parameter not in water_in:
    raise ValueError("water parameter = %s not a property of the water"%water_parameter)

  # if water is not improvable to begin with, but tried to make an improvement
  if not is_improvable:
    return water_in, -2
    
  # water is improvable, but chose an irrelevant parameter
  if not water_in[water_parameter]:
    # returning a reward of +1 here caused the best policy to use a few
    # irrelevant target parameters at first (to ramp up points)
    # and then start installing the relevant elements.
    # This is similar to a salesman who sells useless WTP elements first
    # to ramp up sales, and then sells the right system to close.
    return water_in, 0
  
  # water is improvable, and chose a relevant parameter
  water_out = water_in.copy()
  water_out[water_parameter] = False
  return water_out, +2


# test
print("TDS: High, action: TDS",      act_on_water({"TDS": True }, "TDS"))
print("TDS: Low , action: TDS",      act_on_water({"TDS": False}, "TDS"))
print("TDS: High, action: pipe",      act_on_water({"TDS": True }, None))
print("TDS: Low , action: pipe",      act_on_water({"TDS": False}, None))
try:
  print("TDS: High, action: hardness", act_on_water({"TDS": True }, "hardness"))
except:
  print("TDS: High, action: hardness", "error: parameter not in water")
  
#print("state = 1, action: pipe",      act_on_water(1, None))

TDS: High, action: TDS ({'TDS': False}, 2)
TDS: Low , action: TDS ({'TDS': False}, -2)
TDS: High, action: pipe ({'TDS': True}, -1)
TDS: Low , action: pipe ({'TDS': False}, 1)
TDS: High, action: hardness error: parameter not in water


In [0]:

# Gym env
class WtpDesignerEnv_v0(gym.Env):
    """Water Treatment Plant environment
    
    This is a simulation of a water treatment plant (WTP).
    
    Observation:
      Parameters in water
      Type: Dict of 3 keys, each of which is Discrete(2)
      TDS       True/False (= High/Low)
      Hardness  True/False (= High/Low)
      Bacteria  True/False (= High/Low)
      
    Actions:
      WTP Element to implement at i-th stage
      Type: Discrete(4)
      0 pipe
      2 sediment
      3 softener
      4 uv
      
    Reward: check function "act_on_water"
      
    Episode termination:
      All elements of WTP are chosen
    """
    def __init__(self, attempts_max = 1000):
        self.wtp_elements = [None, "TDS", "hardness", "bacteria"]
        
        
        # number of elements in the WTP system generated
        self.n_elements = 5
        
         # choose the element for the current step
        self.action_space = spaces.Discrete(len(self.wtp_elements))
        
        # https://github.com/openai/gym/blob/master/gym/spaces/dict_space.py
        # the observation space is a Dict of 3 key-value pairs
        # Note that the "FlatDictWrapper" later just flattens this to a single Discrete(8) observation
        # for the sake of being able to use the CategoricalMLPPolicy
        self.observation_space = spaces.Dict({
            "TDS": spaces.Discrete(2), 
            "hardness": spaces.Discrete(2), 
            "bacteria": spaces.Discrete(2),
        })
        
        self.reset()
        #self.seed()

    #def seed(self, seed=None):
    #    self.np_random, seed = seeding.np_random(seed)
    #    return [seed]

    def step(self, action):
        assert self.action_space.contains(action), "action not in action space!"
        assert self.element_i < self.n_elements
        
        # increment number of attempts taken
        self.element_i += 1

        # calculate reward of this element
        wtp_i = self.wtp_elements[action]
        self.state, reward = act_on_water(self.state, wtp_i)
        #print("\t state + element -> state after + reward", wtp_i, self.state, reward_i)
                                      
        # init
        done = False

        # allow a maximum number of attempts to get the WTP selection to work
        if self.element_i >= self.n_elements:
          done = True
          
        return self.state, reward, done, {}
      
    def reset(self, s0=None):
      # s0 - desired state
      if s0 is None:
        s0 = {
          "TDS":  np.random.rand() < 0.5,
          "hardness": np.random.rand() < 0.5,
          "bacteria": np.random.rand() < 0.5,                
        }
      self.state = s0.copy()
      self.element_i = 0
      return self.state

In [3]:

# some smoke testing
print("smoke test")

# example
env_test = WtpDesignerEnv_v0()
state_initial = env_test.reset().copy()
print("water in:")
print("reset()", state_initial)
print("env.state", env_test.state)

print("grid search .. start")
solution = dict(act=None, rew=-99999, water_out=None)
done = False

# builds all possible wtp of these elements
import itertools
wtp_all1 = list(itertools.product(range(len(env_test.wtp_elements)), repeat=env_test.n_elements))
wtp_all2 = []
# append last 2 pipes
for wtp_i in wtp_all1:
  wtp_i = list(wtp_i)
  # 0 is the index of None in env_test.wtp_elements
  wtp_i.append(0)
  wtp_all2.append(wtp_i)

print("grid search .. end")
#print("all wtp")
#print(wtp_all2)

smoke test
water in:
reset() {'TDS': False, 'hardness': False, 'bacteria': True}
env.state {'TDS': False, 'hardness': False, 'bacteria': True}
grid search .. start
grid search .. end


In [4]:

# iterate
for wtp_i in wtp_all2:
  reward_sum = 0
  env_test.reset(s0=state_initial)
  #print("-"*10)
  for action in wtp_i:
    water_out, reward_i, done, _ = env_test.step(action)
    reward_sum += reward_i
    if done:
      break
    
  #print("water in", env_test.state, "wtp", [env_test.wtp_elements[x] for x in wtp_i], "water out", water_out, "reward", reward_sum)
  if reward_sum > solution['rew']:
    solution['act'] = wtp_i
    solution['rew'] = reward_sum
    solution['water_out'] = water_out # last result


print("grid search .. end")

# show result of grid search
print("*"*30)
print("water in:", state_initial)
print("wtp chosen", [env_test.wtp_elements[x] for x in solution['act']])
print("total reward", solution['rew'])
print("water out", solution['water_out'])

grid search .. end
******************************
water in: {'TDS': False, 'hardness': False, 'bacteria': True}
wtp chosen ['bacteria', None, None, None, None, None]
total reward 6
water out {'TDS': False, 'hardness': False, 'bacteria': False}


In [0]:
# register the env with gym
# https://github.com/openai/gym/tree/master/gym/envs#how-to-create-new-environments-for-gym
from gym.envs.registration import register

register(
    id='WtpDesignerEnv-v0',
    #entry_point='gym_foo.envs:FooEnv',
    entry_point=WtpDesignerEnv_v0,
)

# test registration was successful
env = gym.make("WtpDesignerEnv-v0")

In [0]:
# The contents of this cell are mostly copied from garage/examples/...
# NEED TO run this twice for the first run in the runtime in colab, 1st for creating the personal config

from garage.baselines import LinearFeatureBaseline # <<<<<< requires restarting the runtime in colab after the 1st dependency installation above
from garage.envs import normalize
#from garage.envs.box2d import CartpoleEnv # no need since will use WtpDesignerEnv_v0 defined above
from garage.experiment import run_experiment
from garage.tf.algos import TRPO
from garage.tf.envs import TfEnv
#from garage.tf.policies import GaussianMLPPolicy
from garage.tf.policies import CategoricalMLPPolicy

import gym # already imported before

In [0]:
# start a tensorflow session so that we can keep it open after training and use the trained network to see it performing
import tensorflow as tf
sess = tf.InteractiveSession()

# no need to initialize
#sess.run(tf.global_variables_initializer())


In [0]:
class FlattenDictWrapper2(gym.ObservationWrapper):
    """Flattens selected keys of a Dict observation space into
    an array.
    """
    def __init__(self, env, dict_keys):
        super().__init__(env)
        self.dict_keys = dict_keys

        # Figure out observation_space dimension.
        size = 1
        for key in dict_keys:
            shape = self.env.observation_space.spaces[key].n
            size *= shape
        self.observation_space = gym.spaces.Discrete(size)

    def observation(self, observation):
        assert isinstance(observation, dict)
        obs = []
        dims = []
        for key in self.dict_keys:
            obs.append(observation[key])
            dims.append(self.env.observation_space.spaces[key].n)
            
        dims = np.array(dims).cumprod()
        dims = (dims / dims[0]).astype('int')
        
        obs = np.array(obs).astype('int')

        #print("X"*10)
        #print(obs)
        #print(dims)

        return (obs * dims).sum()


In [10]:
# Train the policy (neural network) on the environment
#----------------------------------
from gym import wrappers

# env = TfEnv(normalize(gym.make("CartPole-v0")))
env = gym.make("WtpDesignerEnv-v0")
#env = wrappers.FlattenDictWrapper(env, ["TDS", "hardness", "bacteria"])
wq_elements = ["TDS", "hardness", "bacteria"]
env = FlattenDictWrapper2(env, wq_elements)
env = TfEnv(normalize(env))

policy = CategoricalMLPPolicy(
#policy = DictCategoricalMLPPolicy(
    name="policy", env_spec=env.spec, hidden_sizes=(32, 32))

baseline = LinearFeatureBaseline(env_spec=env.spec)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.random.categorical instead.


In [11]:

algo = TRPO(
    env=env,
    policy=policy,
    baseline=baseline,
    batch_size=4000,
    max_path_length=5+2, #env.n_elements+2, # add 2 since this is just a safety measure
    n_itr=50, # 50 is enough to reach steady state: average return 6.5, max return 8, min return 5
    discount=0.99,
    max_kl_step=0.01,
    plot=False)


Instructions for updating:
Use tf.cast instead.


In [12]:

algo.train(sess=sess)

2019-02-11 14:58:42 | itr #0 | Obtaining samples...
2019-02-11 14:58:42 | itr #0 | Obtaining samples for iteration 0...


0% [##############################] 100% | ETA: 00:00:00

2019-02-11 14:58:43 | itr #0 | Processing samples...



Total time elapsed: 00:00:00


2019-02-11 14:58:43 | itr #0 | Logging diagnostics...
2019-02-11 14:58:43 | itr #0 | Optimizing policy...
2019-02-11 14:58:43 | itr #0 | Computing loss before
2019-02-11 14:58:43 | itr #0 | Computing KL before
2019-02-11 14:58:43 | itr #0 | Optimizing
2019-02-11 14:58:43 | itr #0 | Start CG optimization: #parameters: 1476, #inputs: 800, #subsample_inputs: 800
2019-02-11 14:58:43 | itr #0 | computing loss before
2019-02-11 14:58:43 | itr #0 | performing update
2019-02-11 14:58:43 | itr #0 | computing gradient
2019-02-11 14:58:43 | itr #0 | gradient computed
2019-02-11 14:58:43 | itr #0 | computing descent direction
2019-02-11 14:58:44 | itr #0 | descent direction computed
2019-02-11 14:58:44 | itr #0 | backtrack iters: 1
2019-02-11 14:58:44 | itr #0 | computing loss after
2019-02-11 14:58:44 | itr #0 | optimization finished
2019-02-11 14:58:44 | itr #0 | Computing KL after
2019-02-11 14:58:44 | itr #0 | Computing loss after
2019-02-11 14:58:44 | itr #0 | Fitting baseline...
2019-02-11 1

...

0% [##############################] 100% | ETA: 00:00:00

2019-02-11 14:59:23 | itr #49 | Processing samples...



Total time elapsed: 00:00:00


2019-02-11 14:59:23 | itr #49 | Logging diagnostics...
2019-02-11 14:59:23 | itr #49 | Optimizing policy...
2019-02-11 14:59:23 | itr #49 | Computing loss before
2019-02-11 14:59:23 | itr #49 | Computing KL before
2019-02-11 14:59:23 | itr #49 | Optimizing
2019-02-11 14:59:23 | itr #49 | Start CG optimization: #parameters: 1476, #inputs: 800, #subsample_inputs: 800
2019-02-11 14:59:23 | itr #49 | computing loss before
2019-02-11 14:59:23 | itr #49 | performing update
2019-02-11 14:59:23 | itr #49 | computing gradient
2019-02-11 14:59:23 | itr #49 | gradient computed
2019-02-11 14:59:23 | itr #49 | computing descent direction
2019-02-11 14:59:23 | itr #49 | descent direction computed
2019-02-11 14:59:23 | itr #49 | backtrack iters: 0
2019-02-11 14:59:23 | itr #49 | computing loss after
2019-02-11 14:59:23 | itr #49 | optimization finished
2019-02-11 14:59:23 | itr #49 | Computing KL after
2019-02-11 14:59:23 | itr #49 | Computing loss after
2019-02-11 14:59:24 | itr #49 | Fitting baseli

6.47375

In [0]:
# list of water elements
wq_elements = ["TDS", "hardness", "bacteria"]

In [14]:
# utility function for experiments below
def wqi_to_watersample(wqi):
  # https://stackoverflow.com/a/699891
  #x = "{0:b}".format(wqi)
  x = format(wqi, '03b')
  x = list(x)
  x = x[::-1] # reverse FIXME!?
  #print(x)
  x = [bool(int(y)) for y in x]
  x = dict(zip(wq_elements,x))
  return x

# test
for i in range(8):
  print(i, wqi_to_watersample(i))


0 {'TDS': False, 'hardness': False, 'bacteria': False}
1 {'TDS': True, 'hardness': False, 'bacteria': False}
2 {'TDS': False, 'hardness': True, 'bacteria': False}
3 {'TDS': True, 'hardness': True, 'bacteria': False}
4 {'TDS': False, 'hardness': False, 'bacteria': True}
5 {'TDS': True, 'hardness': False, 'bacteria': True}
6 {'TDS': False, 'hardness': True, 'bacteria': True}
7 {'TDS': True, 'hardness': True, 'bacteria': True}


In [15]:
# Utility function for experiments below

# convert to dicts for readability
#wtp_elements = [None, "TDS", "hardness", "bacteria"]
# rename the elements for readability
wtp_elements = ["pipe", "sand filter", "softener", "UV"]

import pandas as pd

def convert_results_to_df(obs_all, act_all, out_all, rew_all):
    df = []
    for i in range(len(obs_all)):
      in_i = wqi_to_watersample(obs_all[i])
      wtp_i = [wtp_elements[x] for x in act_all[i]]
      out_i = wqi_to_watersample(out_all[i])
      df.append({
          "in_TDS": in_i["TDS"],
          "in_hardness": in_i["hardness"],
          "in_bacteria": in_i["bacteria"],
          "wtp/1": wtp_i[0],
          "wtp/2": wtp_i[1],
          "wtp/3": wtp_i[2],
          "wtp/4": wtp_i[3],
          "wtp/5": wtp_i[4],
          "out_TDS": out_i["TDS"],
          "out_hardness": out_i["hardness"],
          "out_bacteria": out_i["bacteria"],
          "reward": rew_all[i],
      })

    # gather results in pandas dataframe for simplicity of viewing
    df = pd.DataFrame(df)
    df = df[[
          "in_TDS",
          "in_hardness",
          "in_bacteria",
          "wtp/1",
          "wtp/2",
          "wtp/3",
          "wtp/4",
          "wtp/5",
          "out_TDS",
          "out_hardness",
          "out_bacteria",
          "reward",
    ]]

    df = df.sort_values(["in_TDS", "in_hardness", "in_bacteria", "reward"]).set_index(["in_TDS", "in_hardness", "in_bacteria"])
    
    return df

# test
print("TESTING")
convert_results_to_df([0,1,2], [ [0,1,2,0,0], [0,1,2,0,0], [0,1,2,0,0]], [0,1,2], [0,0,0])

TESTING


wtp/1        wtp/2     wtp/3 wtp/4 wtp/5  \
in_TDS in_hardness in_bacteria                                            
False  False       False        pipe  sand filter  softener  pipe  pipe   
       True        False        pipe  sand filter  softener  pipe  pipe   
True   False       False        pipe  sand filter  softener  pipe  pipe   

                                out_TDS  out_hardness  out_bacteria  reward  
in_TDS in_hardness in_bacteria                                               
False  False       False          False         False         False       0  
       True        False          False          True         False       0  
True   False       False           True         False         False       0

# Experiment set 1

In [16]:

n_experiments = 100 # 8 # len(wq_all) # 100
obs_all = [None] * n_experiments
act_all = [None] * n_experiments
rew_all = [None] * n_experiments
out_all = [None] * n_experiments

print("start experiments")
for i in range(n_experiments):
  #print("experiment ", i+1)

  # use random initial states
  state_in = env.reset()

  # use random initial states
  # _ = env.reset()
  # obs_initial = i # wq_all[i]
  # env.state = i # wq_all[i]
  
  # start
  obs_i = state_in
  act_list = []
  rew_sum = 0
  for j in range(5): # env.n_elements
    #action = env.action_space.sample()
    act_i, _ = policy.get_action(obs_i)
    act_list.append(act_i)
    obs_i, rew_i, done, _ = env.step(act_i)
    rew_sum += rew_i
    
    if done: break
    
  obs_all[i] = state_in
  out_all[i] = obs_i
  act_all[i] = act_list
  rew_all[i] = rew_sum

#env.close()

print("done")

start experiments
done


In [0]:
df = convert_results_to_df(obs_all, act_all, out_all, rew_all)

In [18]:
df_1 = df.loc[False, False, False]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

,,,wtp/1,wtp/2,wtp/3,wtp/4,wtp/5,out_TDS,out_hardness,out_bacteria,reward
in_TDS,in_hardness,in_bacteria,,,,,,,,,
False,False,False,pipe,pipe,pipe,pipe,pipe,False,False,False,5.0


In [19]:
df_1 = df.loc[True, False, False]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

,,,wtp/1,wtp/2,wtp/3,wtp/4,wtp/5,out_TDS,out_hardness,out_bacteria,reward
in_TDS,in_hardness,in_bacteria,,,,,,,,,
True,False,False,sand filter,pipe,pipe,pipe,pipe,False,False,False,6.0


In [20]:
df_1 = df.loc[False, True, False]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

,,,wtp/1,wtp/2,wtp/3,wtp/4,wtp/5,out_TDS,out_hardness,out_bacteria,reward
in_TDS,in_hardness,in_bacteria,,,,,,,,,
False,True,False,softener,pipe,pipe,pipe,pipe,False,False,False,6.0


In [21]:
df_1 = df.loc[False, False, True]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

,,,wtp/1,wtp/2,wtp/3,wtp/4,wtp/5,out_TDS,out_hardness,out_bacteria,reward
in_TDS,in_hardness,in_bacteria,,,,,,,,,
False,False,True,UV,pipe,pipe,pipe,pipe,False,False,False,6.0


In [22]:
df_1 = df.loc[True, True, False]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

,,,wtp/1,wtp/2,wtp/3,wtp/4,wtp/5,out_TDS,out_hardness,out_bacteria,reward
in_TDS,in_hardness,in_bacteria,,,,,,,,,
True,True,False,softener,sand filter,pipe,pipe,pipe,False,False,False,7.0


In [23]:
df_1 = df.loc[True, False, True]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

wtp/1        wtp/2 wtp/3 wtp/4 wtp/5  \
in_TDS in_hardness in_bacteria                                               
True   False       True         sand filter           UV  pipe  pipe  pipe   
                   True                  UV  sand filter  pipe  pipe  pipe   

                                out_TDS  out_hardness  out_bacteria  reward  
in_TDS in_hardness in_bacteria                                               
True   False       True           False         False         False     7.0  
                   True           False         False         False     7.0

In [24]:
df_1 = df.loc[False, True, True]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

,,,wtp/1,wtp/2,wtp/3,wtp/4,wtp/5,out_TDS,out_hardness,out_bacteria,reward
in_TDS,in_hardness,in_bacteria,,,,,,,,,
False,True,True,UV,softener,pipe,pipe,pipe,False,False,False,7.0


In [25]:
df_1 = df.loc[True, True, True]
df_1[~df_1.duplicated(["wtp/1", "wtp/2", "wtp/3", "wtp/4", "wtp/5"])]

wtp/1        wtp/2        wtp/3 wtp/4  \
in_TDS in_hardness in_bacteria                                                
True   True        True            softener  sand filter           UV  pipe   
                   True                  UV     softener  sand filter  pipe   
                   True         sand filter           UV     softener  pipe   

                               wtp/5  out_TDS  out_hardness  out_bacteria  \
in_TDS in_hardness in_bacteria                                              
True   True        True         pipe    False         False         False   
                   True         pipe    False         False         False   
                   True         pipe    False         False         False   

                                reward  
in_TDS in_hardness in_bacteria          
True   True        True            8.0  
                   True            8.0  
                   True            8.0

# Experiment set 2

In [26]:

n_experiments2 = 8 # len(wq_all) # 100
obs_all2 = [None] * n_experiments2
act_all2 = [None] * n_experiments2
rew_all2 = [None] * n_experiments2
out_all2 = [None] * n_experiments2

print("start experiments")
for i in range(n_experiments2):
  #print("experiment ", i+1)

  # reset
  s0 = wqi_to_watersample(i)
  obs_initial = env.reset(s0 = s0)

  #print("-"*10)
  #print("obs init", obs_initial)
  
  # start
  obs_i = obs_initial
  act_list = []
  rew_sum = 0
  for j in range(5): # env.n_elements
    #action = env.action_space.sample()
    act_i, _ = policy.get_action(obs_i)
    #print("obs_i", obs_i, "act i", act_i)
    act_list.append(act_i)
    obs_i, rew_i, done, _ = env.step(act_i)
    rew_sum += rew_i
    
    if done: break
    
  obs_all2[i] = obs_initial
  out_all2[i] = obs_i
  act_all2[i] = act_list
  rew_all2[i] = rew_sum

#env.close()

print("done")

start experiments
done


In [0]:
df_deterministic = convert_results_to_df(obs_all2, act_all2, out_all2, rew_all2)

In [28]:
df_deterministic

wtp/1        wtp/2        wtp/3 wtp/4  \
in_TDS in_hardness in_bacteria                                                
False  False       False               pipe         pipe         pipe  pipe   
                   True                  UV         pipe         pipe  pipe   
       True        False           softener         pipe         pipe  pipe   
                   True                  UV     softener         pipe  pipe   
True   False       False        sand filter         pipe         pipe  pipe   
                   True         sand filter           UV         pipe  pipe   
       True        False           softener  sand filter         pipe  pipe   
                   True                  UV     softener  sand filter  pipe   

                               wtp/5  out_TDS  out_hardness  out_bacteria  \
in_TDS in_hardness in_bacteria                                              
False  False       False        pipe    False         False         False   
                   True         pipe    False         False         False   
       True        False        pipe    False         False         False   
                   True         pipe    False         False         False   
True   False       False        pipe    False         False         False   
                   True         pipe    False         False         False   
       True        False        pipe    False         False         False   
                   True         pipe    False         False         False   

                                reward  
in_TDS in_hardness in_bacteria          
False  False       False           5.0  
                   True            6.0  
       True        False           6.0  
                   True            7.0  
True   False       False           6.0  
                   True            7.0  
       True        False           7.0  
                   True            8.0